In [3]:
import matplotlib.pyplot as plt
import cv2
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import time
from multiprocessing import Pool
from functools import partial

from sklearn.model_selection import train_test_split

In [4]:
dataset_dir = "C:/Users/frede/Desktop/Bachelorprojekt/datasets/VALID-dataset"

In [5]:
#define a custom dataset class
class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.Tensor(self.data[idx])
        y = torch.Tensor(self.targets[idx])
        
        return x,y

In [6]:
def find_image_from_json(json_path):
    f = open(json_path)
    json_file = json.load(f)
    path_to_image_file = json_file["file_name"]
    
    return(f"C:/Users/frede/Desktop/Bachelorprojekt/datasets/VALID-dataset/{path_to_image_file}")

def find_image_from_json_simple(json_path):
    f = open(json_path)
    json_file = json.load(f)
    path_to_image_file = json_file["file_name"]
    #print(path_to_image_file)
    
    return(os.path.join(dataset_dir,path_to_image_file))

In [7]:
device = torch.device("cuda")
tens1 = torch.randn(10,3,1000,1000)
tens2 = torch.randn(10,1000,1000)

In [8]:
#input size: amount_img, channels, width, height
def resize_256_256(inp):
    return(F.interpolate(inp, size=(256,256), mode="bilinear", align_corners=False))
        

In [9]:
dataset_dir = "C:/Users/frede/Desktop/Bachelorprojekt/datasets/VALID-dataset"

In [11]:
def create_data():
    start_time = time.time()

    images_list = []
    depth_images_list = []
    #transform
    for tup in zip(os.listdir(os.path.join(dataset_dir,"depth")), os.listdir(os.path.join(dataset_dir,"label"))):
        
        # Path to image and depth image
        current_image_path = find_image_from_json_simple(os.path.join(dataset_dir, "label", tup[1]))
        current_depth_image_path = os.path.join(dataset_dir,"depth", tup[0])

        # Read in image and depth image
        image = cv2.imread(current_image_path)/256
        depth_image = cv2.imread(current_depth_image_path)/256
        
        
        #print(depth_image.shape)
        # Convert image and depth image to tensors: shape from (1024,1024,3) to (1,3,1024,1024) -> (256,256,3) to (1,3,256,256)
        image = resize_256_256(torch.from_numpy(image).unsqueeze(0).permute(0,3,1,2))
        
        # Convert image and depth image to tensors: shape from (1024,1024,3) to (1,3,1024,1024) -> (256,256,3) to (1,3,256,256) to (1,256,256)
        depth_image = resize_256_256(torch.from_numpy(depth_image).unsqueeze(0).permute(0,3,1,2))[:,0,:,:]
        

        # Add image tensor and depth image tensor to lists
        images_list.append(image)
        depth_images_list.append(depth_image)
        

           
    tensor_images = torch.cat(images_list,dim=0)
    tensor_depth_images = torch.cat(depth_images_list, dim=0)
    
    elapsed_time = time.time() - start_time
    return(tensor_images,tensor_depth_images)
train_data = create_data()


KeyboardInterrupt: 